In [51]:
#Priyanshu Panchal 210968174
import numpy as np
import pandas as pd

In [19]:
df = pd.read_csv("Ads_Clicks.csv")
df.head()

,Ad 1,Ad 2,Ad 3,Ad 4,Ad 5,Ad 6,Ad 7,Ad 8,Ad 9,Ad 10
0,1,0,0,0,1,0,0,0,1,0
1,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0


The problem agent formulation involves determining the most optimal ad to display to a user at a given time instant to maximize the number of clicks.

The MAB agent's objective is to learn the true CTR of each ad while minimizing the regret, which is the difference between the expected number of clicks obtained by displaying the best ad and the expected number of clicks obtained by displaying the chosen ad at each time step. The MAB agent must balance the exploration of less-known ads to learn their CTRs with the exploitation of the ads that are known to have higher CTRs to maximize the total number of clicks.

In [20]:
def reward(t,A):
  ad = 'Ad {}'.format(A+1)
  reward = df.loc[df.index==t, ad]
  return reward.values[0]

In [30]:
num_steps = 2000
k=10

In [31]:
prob = np.random.random(size=num_steps)


In [43]:
def eps(e):
  tot_reward = 0
  N={}
  Q={}
  for i in range(k):
    N[i] = 0
    Q[i] = 0
  for t in range(num_steps):
    if prob[t]>e:
      A=max(zip(Q.values(),Q.keys()))[1] #exploitation
    else:
      A=np.random.randint(k) #exploration

    R = reward(t, A)
    N[A] = N[A]+1
    Q[A] = Q[A] + (R- Q[A])/N[A]
    tot_reward += R
  return tot_reward

In [44]:
print(eps(0.01))

249


In [45]:
print(eps(0.3))

357



As we can see, the setting with a higher value of epsilon has a higher cummulative reward function value. This is because, higher the value of epsilon, higher is the exploration factor, hence, even though there are sub optimal payouts initially, in the long run, we have a higher probability of hitting the jackpot (in this case, picking an arm with higher reward).

The ε-greedy approach estimates the value of an action using the sample average method, which calculates the average of rewards received for that action. However, this method can have high variance if the number of samples is low, resulting in slow convergence to true values. To address this, the ε-greedy approach explores more at the beginning to reduce uncertainty and exploits the best action based on estimated values later.

In [46]:
np.seterr(divide='ignore', invalid='ignore')
c=1.5

In [49]:
def ucb(e):
  tot_reward = 0
  N={}
  R={}
  for i in range(k):
    N[i] = 1
    R[i] = 0

  for t in range(num_steps):
    q = {}
    delta = {}
    ucb = {}

    for i in range(k):
      q[i] = R[i]/N[i] # average reward till timestep t
      delta[i] = np.sqrt(c*np.log(t)/N[i]) # confidence interval
      ucb[i] = q[i] + delta[i]

    # updating values of par+ameters
    A = max(zip(ucb.values(), ucb.keys()))[1]
    N[A] = N[A] + 1
    R[A] = R[A] + reward(t, A)
    tot_reward += reward(t, A) # computing total reward
  return tot_reward

In [50]:
print(ucb(1.5))

338


The Upper-Confidence-Bound approach estimates action values using the Upper Confidence Bound (UCB), which combines the average reward and an uncertainty term. This term prioritizes actions that haven't been selected many times and gives lower priority to those that have. This approach results in more stable value estimates and faster convergence to the optimal action.